In [0]:

import warnings
warnings.filterwarnings('ignore')
!pip install selenium fuzzywuzzy
#Imports
import os
import re
import sys
import nltk
import spacy
import sqlite3
import selenium
import numpy as np
import pandas as pd



from bs4 import BeautifulSoup
from selenium import webdriver
from fuzzywuzzy import fuzz
from multiprocessing import Pool
from IPython.display import display,HTML
from sklearn.feature_extraction.text import *
from selenium.webdriver.chrome.options import Options
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle


#nlp = spacy.load("en_core_web_lg")


In [0]:
#Setting chrome driver path
CHROMEDRIVER_PATH="C:/driver/chromedriver.exe"

In [0]:
#creating db
frame = 'dictionary_ms'
sql_transaction = []

def create_table():
    connection = sqlite3.connect('{}.db'.format(frame))
    c = connection.cursor()
    c.execute("CREATE TABLE IF NOT EXISTS table_data(mcr_id INT PRIMARY KEY, data TEXT)")
    return connection,c

def inject_data(cur_func,data_corpus):
    sql_query = '''INSERT INTO table_data(mcr_id,data) VALUES(?,?) '''
    cur_func.execute(sql_query, data_corpus)
    print(cur_func.lastrowid)
    return cur_func.lastrowid


In [0]:
def set_options_driver(headless=True,window_size=(1440,900)):
    option_chrome=Options()
    prefs = {"translate":{"enabled":"true"}}
    option_chrome.add_experimental_option("prefs", prefs)
    option_chrome.headless=headless
    driver=webdriver.Chrome(options=option_chrome,executable_path=CHROMEDRIVER_PATH)
    driver.set_window_size(1440, 900) #If  windowsize is not set ,driver will give not able to interact.Error in headless mode
    return driver




def extract(addr=None):
    #setting local variable ty to None because it would or wouldn't be populated based on scrapping results
    ty=None 
    driver=set_options_driver(headless=True)
    driver.get('https://www.google.com/')
    driver.find_element_by_css_selector('input.gLFyf.gsfi').send_keys(addr)
    driver.find_element_by_css_selector('input[name=btnK]').submit()
    
    try:
        #If spelling correction is suggested by google api we click on it in order to correct the spelling
        driver.find_element_by_css_selector('a.gL9Hy').click() 
    
    except:
        pass
    
    try:
        
        #Here we are trying to find the exact details shared on google maps 
        ty=driver.find_elements_by_css_selector('span.YhemCb')
        ty=[i.text.replace('\r', ' ').replace('\n', ' ') for i in ty]
        add=[i.text.replace('\r', ' ').replace('\n', ' ') for i in driver.find_elements_by_css_selector('span.ggV7z')]
        ty+=add
        
       
    except:
        pass
    
    
    headers=list(filter(lambda x:len(x)>0,[i.text.replace('\r', ' ').replace('\n', ' ') for  i in driver.find_elements_by_css_selector('h3.LC20lb')]))
    headers=" ".join(i for i in headers)
    headers=list(filter(lambda x:len(x)>0,headers.split(" ")))
    return_head=headers
    headers=headers*3
    
    if ty:
        ty=ty*200 #giving additional weight 
        ty=" ".join(i for i in ty)
    else:
        ty=""


    
    
    res=driver.find_elements_by_css_selector('div.r')
    res=[i.find_elements_by_tag_name('a') for i in res]
    res=[i[0].get_attribute('href') for i in res]
    
    links=list(filter(lambda x:len(x)!=0,res))
    
    span_el=driver.find_elements_by_css_selector('span.st')
    front_corpus=[i.text.replace('\r', ' ').replace('\n', ' ') for i in span_el]
    texter=[i.strip() for i in front_corpus]
    pattern=re.compile(r"[^a-zA-Z-&']")
    subheads= " ".join(i for i in [" ".join(i for i in re.sub(pattern," ",i).split(" ") if len(i)!=0) for i in texter])
    


    corpus=[]
    list_el=[]
    list_yl=[]
    for ii in links:
        #print(ii)
        
        
        corpuss=[]
        try:
            
            driver.get(ii)
            
            try:
                list_el+=[driver.find_element_by_css_selector('span.category-str-list').text.replace('\r', ' ').replace('\n', ' ')]*10
            except:
                pass



            try:
                list_yl+=[driver.find_element_by_css_selector('dd.categories').text.replace('\r', ' ').replace('\n', ' ')]*10
            except:
                pass


            temp_cache=driver.find_element_by_tag_name("body").text.replace('\r', ' ').replace('\n', ' ')
            html=driver.page_source
            soup=BeautifulSoup(html)
            
            results=soup.findAll('p')
            for j in results:
                corpuss+=(j.text.replace('\r', ' ').replace('\n', ' ').split()) #words+=j.text.strip().split(" ")
            corpuss+=temp_cache.split()
            
            corpus+=corpuss
            #print(len(corpus))

        except:
            print("Exec--> ",i)
            pass
    driver.close()
    
    #removing trailing spaces and new line characters
    corpus=[i for i in corpus]
    #print("final",len(corpus))
    
    #Any word having length less than 3 would be removed from the corpus
    #corpus=list(filter(lambda x: len(x)>2,corpus))[:100000]
    list_el=" ".join(i for i in list_el)
    list_yl=" ".join(i for i in list_yl)
    
  

    #All the information scrapped from different  links is merged into one para
    
    
    corpus=subheads+" "+ty+" "+list_el+" "+list_yl+" "+" ".join(i for i in corpus)
    
    
    
   
    return corpus


In [0]:
list=["svm","knn","random forest","xgboost","light gbm","cnn","rnn","lstm","attention lstm","object detection","ssd","faster rcnn"]

for idx,i in enumerate(list):
      
    corpus=extract(i)
    sql_transaction.append([idx,corpus])
    
        
  if len(sql_transaction)==5:
      connector,cur=create_table()
      with connector:
          for i in sql_transaction:
              inject_data(cur,i)
          sql_transaction=[]
          connector.commit()


  